# División del conjunto de datos
En este notebook, nos enfocaremos en la división del conjunto de datos oceanográficos en conjuntos de entrenamiento, validación y prueba. Es crucial que esta división respete las estaciones del año, asegurando que cada conjunto contenga ejemplos de todas las estaciones sin solapamientos temporales. Esto garantizará que nuestros modelos de aprendizaje profundo puedan generalizar adecuadamente a datos no vistos, manteniendo la estacionalidad de los fenómenos oceanográficos.

In [6]:
import numpy as np
import xarray as xr
import torch

dataset = xr.open_dataset("D://Aaron//cmems_mod_glo_phy_my_0.083deg_P1D-m_6years_thetao_v3.nc")
lsm = xr.open_dataset("D://Aaron//datos_mascara.nc")

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## 1.Preparación de las estaciones del año

Asignamos a cada instancia temporal su estación correspondiente para facilitar una división equilibrada del conjunto de datos.

In [8]:
# Función para asignar estación según el mes
def get_season(month):
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    else:
        return 'fall'

# Aplicar la función get_season al dataset
dataset['season'] = xr.apply_ufunc(
    np.vectorize(get_season),
    dataset['time'].dt.month,
    vectorize=True,
    dask="parallelized",
    output_dtypes=[str]
)


## 2.Función para dividir el dataset

Definimos una función que divide el dataset según proporciones especificadas y verifica que cada conjunto tenga ejemplos de todas las estaciones.

In [9]:
def split_by_time(dataset, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    """
    Divide el dataset en entrenamiento, validación y prueba asegurando que todas las estaciones estén
    representadas en cada conjunto y no haya solapamientos temporales.

    Args:
        dataset (xr.Dataset): El dataset con la dimensión 'time'.
        train_ratio (float): Proporción de datos para el conjunto de entrenamiento.
        val_ratio (float): Proporción de datos para el conjunto de validación.
        test_ratio (float): Proporción de datos para el conjunto de prueba.

    Returns:
        train_dataset (xr.Dataset): Conjunto de entrenamiento con todas las estaciones.
        val_dataset (xr.Dataset): Conjunto de validación con todas las estaciones.
        test_dataset (xr.Dataset): Conjunto de prueba con todas las estaciones.
    """
    # Total de puntos temporales en el dataset
    num_times = len(dataset['time'])
    
    # Calcular los índices para dividir los datos
    train_index = int(train_ratio * num_times)
    val_index = int(val_ratio * num_times)
    
    # Crear los conjuntos secuencialmente por tiempo (sin solapamiento)
    train_dataset = dataset.isel(time=slice(0, train_index))
    val_dataset = dataset.isel(time=slice(train_index, train_index + val_index))
    test_dataset = dataset.isel(time=slice(train_index + val_index, num_times))

    # Asegurar que cada conjunto tenga ejemplos de todas las estaciones
    def ensure_all_seasons(dataset):
        seasons_present = np.unique(dataset['season'].values)
        missing_seasons = set(['winter', 'spring', 'summer', 'fall']) - set(seasons_present)
        if missing_seasons:
            raise ValueError(f"El dataset no tiene datos de las estaciones: {missing_seasons}")
        return dataset
    
    train_dataset = ensure_all_seasons(train_dataset)
    val_dataset = ensure_all_seasons(val_dataset)
    test_dataset = ensure_all_seasons(test_dataset)

    return train_dataset, val_dataset, test_dataset


## 3.Aplicación de la función de división

Dividimos el dataset utilizando las proporciones estándar de 70% para entrenamiento, 15% para validación y 15% para prueba.

In [10]:
# Aplicar la función para dividir el dataset
train_dataset, val_dataset, test_dataset = split_by_time(dataset)


## 4.Verificación de la división

Es fundamental verificar que la división se haya realizado correctamente, tanto en términos de tamaños como en la representación de todas las estaciones.

### Tamaños de los conjuntos

In [11]:
# Imprimir tamaños de los conjuntos
print(f"Conjunto de entrenamiento: {len(train_dataset['time'])} instancias temporales")
print(f"Conjunto de validación: {len(val_dataset['time'])} instancias temporales")
print(f"Conjunto de prueba: {len(test_dataset['time'])} instancias temporales")


Conjunto de entrenamiento: 1790 instancias temporales
Conjunto de validación: 383 instancias temporales
Conjunto de prueba: 385 instancias temporales


### Rangos de tiempo de cada conjunto

In [12]:
# Imprimir rangos de tiempo de cada conjunto
print(f"Rango de tiempo del conjunto de entrenamiento: {train_dataset['time'].values[0]} a {train_dataset['time'].values[-1]}")
print(f"Rango de tiempo del conjunto de validación: {val_dataset['time'].values[0]} a {val_dataset['time'].values[-1]}")
print(f"Rango de tiempo del conjunto de prueba: {test_dataset['time'].values[0]} a {test_dataset['time'].values[-1]}")


Rango de tiempo del conjunto de entrenamiento: 2014-01-01T00:00:00.000000000 a 2018-11-25T00:00:00.000000000
Rango de tiempo del conjunto de validación: 2018-11-26T00:00:00.000000000 a 2019-12-13T00:00:00.000000000
Rango de tiempo del conjunto de prueba: 2019-12-14T00:00:00.000000000 a 2021-01-01T00:00:00.000000000


Los conjuntos están ordenados cronológicamente sin solapamientos, lo que evita fugas de información temporal entre ellos.

## 5.Verificación de la representación de estaciones
Nos aseguramos de que cada conjunto contenga datos de todas las estaciones del año.

In [13]:
# Función para verificar la presencia de todas las estaciones
def check_seasons(dataset):
    seasons = set(dataset['season'].values)
    missing_seasons = set(['winter', 'spring', 'summer', 'fall']) - seasons
    if missing_seasons:
        print(f"Faltan las siguientes estaciones: {missing_seasons}")
    else:
        print("Todas las estaciones están representadas.")

# Verificar en cada conjunto
print("Conjunto de entrenamiento:")
check_seasons(train_dataset)

print("\nConjunto de validación:")
check_seasons(val_dataset)

print("\nConjunto de prueba:")
check_seasons(test_dataset)


Conjunto de entrenamiento:
Todas las estaciones están representadas.

Conjunto de validación:
Todas las estaciones están representadas.

Conjunto de prueba:
Todas las estaciones están representadas.


La verificación confirma que cada conjunto incluye datos de invierno, primavera, verano y otoño, lo cual es esencial para que el modelo aprenda patrones estacionales y generalice correctamente.

## Conclusión
Hemos realizado una división cuidadosa del conjunto de datos, asegurando que cada subconjunto (entrenamiento, validación y prueba) contenga ejemplos de todas las estaciones del año y que no existan solapamientos temporales entre ellos. Esta estrategia es fundamental para entrenar modelos robustos y confiables en el contexto de datos temporales y estacionales, como los oceanográficos